In [1]:
import numpy as np
from pygeotools import *
from pygeopinn import *

In [2]:
pygeo = pygeotools()

pygeotools was initialized with `verbose=True`.


In [3]:
pygeo.loadModel("covobs", "covobs_hdf5", "covobs.hdf5")

pygeo.setGrid("1deg")

context = {
    "r": pygeo.constants["rCore"],
    "lmax": 13
}

MF = pygeo.addMeasure("covobs", "MF", context)
SV = pygeo.addMeasure("covobs", "SV", context)

13


In [4]:
pinn = pygeopinn(nb_layers=3, nb_nodes=32)

In [5]:
_, (thetas, phis) = pygeo.getCurrentGrid()

thetas = np.rad2deg(thetas)
phis = np.rad2deg(phis)

In [6]:
theta_min, theta_max = 20, 60
phi_min, phi_max = 20, 60

idx_theta_min = np.argmin(np.abs(thetas - theta_min))
idx_theta_max = np.argmin(np.abs(thetas - theta_max))
idx_phi_min = np.argmin(np.abs(phis - phi_min))
idx_phi_max = np.argmin(np.abs(phis - phi_max))

In [7]:
Br = pygeo.selectFromMeasure("covobs", MF, options={"time": 2020, "component": "r"})
dBrdt = pygeo.selectFromMeasure("covobs", SV, options={"time": 2020, "component": "r"})
times = pygeo.selectFromQuantity("covobs", "times")

In [8]:
pinn.set_grid(thetas[idx_theta_min:idx_theta_max], phis[idx_phi_min:idx_phi_max])

In [9]:
pinn.add("Br", Br[idx_theta_min:idx_theta_max, idx_phi_min:idx_phi_max])
pinn.add("dBrdt", dBrdt[idx_theta_min:idx_theta_max, idx_phi_min:idx_phi_max])
pinn.add("times", times)

In [11]:
pinn.init()

pinn.train(nb_reals=1, nb_epochs=1000)

dBrdt_pred, uH_pred = pinn.results()

Epoch 0: L1 = 22970.41796875, L2 = 0.000792569131590426
Epoch 100: L1 = 20281.994140625, L2 = 0.000778063025791198
Epoch 200: L1 = 17082.474609375, L2 = 0.000785222218837589
Epoch 300: L1 = 14042.3447265625, L2 = 0.0007906733662821352
Epoch 400: L1 = 11444.7529296875, L2 = 0.0007961515220813453
Epoch 500: L1 = 9434.5849609375, L2 = 0.0008014122140593827
Epoch 600: L1 = 7984.4150390625, L2 = 0.0008058748207986355
Epoch 700: L1 = 6966.44677734375, L2 = 0.0008093152428045869
Epoch 800: L1 = 6232.05517578125, L2 = 0.0008118139812722802
Epoch 900: L1 = 5672.21630859375, L2 = 0.0008136616670526564


In [ ]:
dBrdt_pred

(40, 40)